<a href="https://colab.research.google.com/github/vifirsanova/empi/blob/main/SCENARIOS/gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import json

class Encoder():
  def __init__(self, block, prompt):
    self.prompt = prompt
    self.block = block
  def tokenizer(self):
    tokenized_prompt = self.prompt.split()
    return tokenized_prompt
  def cypher(self, tokenized_prompt):
    cyphered_prompt = tokenized_prompt
    return cyphered_prompt
  def get_info(self, cyphered_prompt):
    prompt_info = cyphered_prompt
    return prompt_info
  def update_blockchain(self, current_block, cyphered_prompt, prompt_info):
    current_block = [cyphered_prompt, prompt_info]
    return current_block
  def save_block(self):
    tokenized_prompt = self.tokenizer()
    cyphered_prompt = self.cypher(tokenized_prompt)
    prompt_info = self.get_info(cyphered_prompt)
    current_block = self.update_blockchain(self.block, cyphered_prompt, prompt_info)
    user_block = current_block # to json
    return user_block

class Decoder():
  def __init__(self, current_block):
    self.current_block = current_block
  def block2chain(self):
    vectorized_block = self.current_block
    return vectorized_block
  def language_model(self, vectorized_block):
    lm_outputs = [vectorized_block, vectorized_block]
    return lm_outputs
  def privacy_consistency_check(self, lm_outputs):
    privacy_scores = {}
    consistency_scores = {}
    for output in lm_outputs[0][0]:
      privacy_scores[output] = 1
      consistency_scores[output] = 1
    return privacy_scores, consistency_scores
  def human_evaluation(self, lm_outputs):
    preference_scores = {}
    for output in lm_outputs[0][0]:
      preference_scores[output] = 1
    return preference_scores
  def argmax(self, privacy_scores, consistency_scores, preference_scores):
    best_model_response = privacy_scores, consistency_scores, preference_scores
    return best_model_response
  def model_output(self):
    vectorized_block = self.block2chain()
    lm_outputs = self.language_model(vectorized_block)
    privacy_scores, consistency_scores = self.privacy_consistency_check(lm_outputs)
    preference_scores = self.human_evaluation(lm_outputs)
    best_model_response = self.argmax(privacy_scores, consistency_scores, preference_scores)
    outputs = best_model_response # text only
    return best_model_response

def modeling(block, prompt):
  current_block = Encoder(block, prompt).save_block()
  result = Decoder(current_block).model_output()
  return result

#!wget -O block.json https://raw.githubusercontent.com/vifirsanova/empi/main/SCENARIOS/%D0%90%D0%BD%D1%8F_block(1).json

#with open('block.json', 'r') as f:
#  block = json.load(f)

modeling()

check the function


({'check': 1, 'the': 1, 'function': 1},
 {'check': 1, 'the': 1, 'function': 1},
 {'check': 1, 'the': 1, 'function': 1})